In [30]:
import pandas as pd
import os
import datetime
import json
import numpy as np

In [31]:
ROOT_DIR = r"C:\dev\backend\tasks\samples\cmots"
asof_date = datetime.date(2022, 12, 7)
DIR_PATH = os.path.join(ROOT_DIR, asof_date.strftime("%d%m%Y"))

DIR_PATH

'C:\\dev\\backend\\tasks\\samples\\cmots\\07122022'

In [37]:
def get_df(dir_path, file_name, asof_date, lst_col, lst_drop_col):
    file_name = os.path.join(dir_path, file_name)
    print(file_name)
    if os.path.isfile(file_name):
        print('Reading the file into df...')
        df = pd.read_csv(file_name, sep ='|', names=lst_col, encoding= 'unicode_escape' )
        df.iloc[:, -1] = df.iloc[:, -1].str.split('<</EOR>>', expand=True)
        # print(df.head())
        df = df.drop(lst_drop_col, axis=1)
    else:
        df = pd.DataFrame()
    return df

In [41]:
lst_col = ["CO_CODE","WDATE","INDEXCODE","CLOSEPRICE","NOOFSHARES",
           "FULLMCAP","FF_ADJFACTOR","FF_MCAP","WEIGHT_INDEX","FLAG"]
lst_used = ["CO_CODE","WDATE","CLOSEPRICE","NOOFSHARES",
           "FULLMCAP","FF_ADJFACTOR","FF_MCAP","FLAG"]
lst_drop = [item for item in lst_col if item not in lst_used]


df_bseweight = get_df(DIR_PATH, "bseweight.csv", asof_date, lst_col, lst_drop_col)
df_bseweight = df_bseweight[df_bseweight.FLAG != 'D'] # remove records with FLAG == D
#print(df_bseweight.head())

# ------------------------------------------------
df_nseweight = get_df(DIR_PATH, "nseweight.csv", asof_date, lst_col, lst_drop_col)
df_nseweight = df_nseweight[df_nseweight.FLAG != 'D'] # remove records with FLAG == D
#print(df_nseweight.head())

C:\dev\backend\tasks\samples\cmots\07122022\bseweight.csv
Reading the file into df...
C:\dev\backend\tasks\samples\cmots\07122022\nseweight.csv
Reading the file into df...


In [43]:
total_mkt_cap = df_bseweight["FULLMCAP"].sum()
df_bseweight.sort_values("FULLMCAP", ascending=False, inplace=True)
df_bseweight.drop_duplicates(inplace=True)
#df_bseweight.reset_index(inplace=True)
df_bseweight['CUM_TOTAL'] = df_bseweight['FULLMCAP'].cumsum()
#df_bseweight['CUM_PERC'] = 100*df_bseweight['CUM_TOTAL']/total_mkt_cap

print('Total Mkt Cap : ',0.7 *total_mkt_cap)

df_bseweight["MCAP"] = np.where(df_bseweight['CUM_TOTAL'] < (0.7 * total_mkt_cap) , "Largecap", np.nan)
df_bseweight["MCAP"] = np.where((df_bseweight['CUM_TOTAL'] > (0.7 * total_mkt_cap)) & (df_bseweight['CUM_TOTAL'] < (0.9 * total_mkt_cap)), "Midcap", np.nan)
df_bseweight["MCAP"] = np.where(df_bseweight['CUM_TOTAL'] > (0.9 * total_mkt_cap) , "SmallCap", np.nan)

df_bseweight

Total Mkt Cap :  19794843.782


,index,CO_CODE,WDATE,CLOSEPRICE,NOOFSHARES,FULLMCAP,FF_ADJFACTOR,FF_MCAP,FLAG,CUM_TOTAL,MCAP
0,980,476,07/12/2022,2649.90,6765450384,1792776.70,0.5079,910546.879047,A,1792776.70,nan
1,2869,5400,07/12/2022,3380.60,3659051373,1236978.91,0.2770,342661.399679,A,3029755.61,nan
2,2812,4987,07/12/2022,1609.95,5575921736,897695.52,0.7909,709989.677655,A,3927451.13,nan
3,2180,2806,07/12/2022,1605.05,4208080843,675418.02,0.8656,584651.710124,A,4602869.15,nan
4,2879,5418,07/12/2022,922.10,6975150150,643178.60,0.9980,641899.717621,A,5246047.75,nan
...,...,...,...,...,...,...,...,...,...,...,...
1235,6232,76037,07/12/2022,27.00,3153000,8.51,0.4758,4.049465,A,28278322.24,SmallCap
1236,6264,76397,01/12/2022,38.50,2088000,8.04,0.2672,2.148621,A,28278330.28,SmallCap
1237,6277,76663,07/12/2022,8.30,7853000,6.52,0.7273,4.742318,A,28278336.80,SmallCap
1238,6239,76237,02/12/2022,30.25,2100000,6.35,0.4318,2.741990,A,28278343.15,SmallCap
